In [8]:
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import datetime as dt
from scipy.optimize import minimize 

Get data

In [9]:
# Define the start and end dates of stock prices
def get_data(name, start,end):
    # download data 
    df = web.DataReader(name, 'yahoo', start, end)
    df = df.sort_values(by='Date')
    return df

In [10]:
start_date = dt.datetime(2011, 1, 1)
end_date = dt.datetime(2022, 9, 1)
GM = get_data("GM", start=start_date, end=end_date)
VIX = get_data("^VIX", start=start_date, end=end_date)
SP = get_data("^GSPC", start=start_date, end=end_date)
NASDAQ = get_data("^IXIC", start=start_date, end=end_date)
GOLD = get_data("GC=F", start=start_date, end=end_date)
OIL = get_data("CL=F", start=start_date, end=end_date)

In [11]:
target_move = GM[["Adj Close"]].pct_change(1)
GM_Volume = GM[["Volume"]]
VIX = VIX[["Close"]]
SP = SP[["Adj Close"]]
NASDAQ = NASDAQ[["Adj Close"]]
GOLD = GOLD[["Adj Close"]]
OIL = OIL[["Adj Close"]]
rolling_10 = GM[['Adj Close']].rolling(10).mean()

In [12]:

df = GM_Volume.merge(VIX,how = "left",left_index=True, right_index=True).merge(SP,left_index=True, right_index=True).merge(NASDAQ,left_index=True, right_index=True).merge(GOLD,left_index=True, right_index=True).merge(OIL,left_index=True, right_index=True).merge(rolling_10,left_index=True, right_index=True).merge(target_move,left_index=True, right_index=True)
df.columns = ["GM Volume","VIX","SP","NASDAQ","GOLD","OIL","rolling_10","target_move",]
df

/var/folders/4_/0p3l6m0x22qdsjwms8cj48sm0000gn/T/ipykernel_3068/3374355406.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = GM_Volume.merge(VIX,how = "left",left_index=True, right_index=True).merge(SP,left_index=True, right_index=True).merge(NASDAQ,left_index=True, right_index=True).merge(GOLD,left_index=True, right_index=True).merge(OIL,left_index=True, right_index=True).merge(rolling_10,left_index=True, right_index=True).merge(target_move,left_index=True, right_index=True)
/var/folders/4_/0p3l6m0x22qdsjwms8cj48sm0000gn/T/ipykernel_3068/3374355406.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = GM_Volume.merge(VIX,how = "left",left_index=True, right_index=True).merge(SP,left_index=True, right_index=True).merge(NASDAQ,left_index=True, right_index=

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10,target_move
Date,,,,,,,,
2011-01-03,24874900.0,17.610001,1271.869995,2691.520020,1422.599976,91.550003,NaN,NaN
2011-01-04,32363400.0,17.379999,1270.199951,2681.250000,1378.500000,89.379997,NaN,0.022666
2011-01-05,22503900.0,17.020000,1276.560059,2702.199951,1373.400024,90.300003,NaN,0.004485
2011-01-06,38556900.0,17.400000,1273.849976,2709.889893,1371.400024,88.379997,NaN,0.021802
2011-01-07,19901100.0,17.139999,1271.500000,2703.169922,1368.500000,88.029999,NaN,0.002056
...,...,...,...,...,...,...,...,...
2022-08-26,16179500.0,25.559999,4057.659912,12141.709961,1736.099976,93.059998,39.029891,-0.028720
2022-08-29,13408900.0,26.209999,4030.610107,12017.669922,1736.599976,97.010002,39.040865,0.007137
2022-08-30,15090600.0,26.209999,3986.159912,11883.139648,1723.199951,91.639999,39.063747,-0.007357


Preprocessing

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [14]:
df.loc[df["target_move"] > 0, "direction"] = int(1)
df.loc[df["target_move"] < 0, "direction"] = int(0)
df

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10,target_move,direction
Date,,,,,,,,,
2011-01-03,24874900.0,17.610001,1271.869995,2691.520020,1422.599976,91.550003,NaN,NaN,NaN
2011-01-04,32363400.0,17.379999,1270.199951,2681.250000,1378.500000,89.379997,NaN,0.022666,1.0
2011-01-05,22503900.0,17.020000,1276.560059,2702.199951,1373.400024,90.300003,NaN,0.004485,1.0
2011-01-06,38556900.0,17.400000,1273.849976,2709.889893,1371.400024,88.379997,NaN,0.021802,1.0
2011-01-07,19901100.0,17.139999,1271.500000,2703.169922,1368.500000,88.029999,NaN,0.002056,1.0
...,...,...,...,...,...,...,...,...,...
2022-08-26,16179500.0,25.559999,4057.659912,12141.709961,1736.099976,93.059998,39.029891,-0.028720,0.0
2022-08-29,13408900.0,26.209999,4030.610107,12017.669922,1736.599976,97.010002,39.040865,0.007137,1.0
2022-08-30,15090600.0,26.209999,3986.159912,11883.139648,1723.199951,91.639999,39.063747,-0.007357,0.0


In [15]:
df = df.dropna()
df

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10,target_move,direction
Date,,,,,,,,,
2011-01-14,5894200.0,15.460000,1293.239990,2755.300049,1360.400024,91.540001,29.558305,-0.001829,0.0
2011-01-18,8918200.0,15.870000,1295.020020,2765.850098,1368.099976,91.379997,29.633104,-0.004450,0.0
2011-01-19,12667700.0,17.309999,1281.920044,2725.360107,1370.199951,90.860001,29.594548,-0.016565,0.0
2011-01-20,15844900.0,17.990000,1280.260010,2704.290039,1346.500000,88.860001,29.525918,-0.005883,0.0
2011-01-21,9548500.0,18.469999,1283.349976,2689.540039,1341.000000,89.110001,29.397909,0.001614,1.0
...,...,...,...,...,...,...,...,...,...
2022-08-26,16179500.0,25.559999,4057.659912,12141.709961,1736.099976,93.059998,39.029891,-0.028720,0.0
2022-08-29,13408900.0,26.209999,4030.610107,12017.669922,1736.599976,97.010002,39.040865,0.007137,1.0
2022-08-30,15090600.0,26.209999,3986.159912,11883.139648,1723.199951,91.639999,39.063747,-0.007357,0.0


Use n-1 date data to predict n day data. So remove last day of predicted variables and first day 

In [16]:
X = df.drop(columns=["direction", "target_move"])
X

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10
Date,,,,,,,
2011-01-14,5894200.0,15.460000,1293.239990,2755.300049,1360.400024,91.540001,29.558305
2011-01-18,8918200.0,15.870000,1295.020020,2765.850098,1368.099976,91.379997,29.633104
2011-01-19,12667700.0,17.309999,1281.920044,2725.360107,1370.199951,90.860001,29.594548
2011-01-20,15844900.0,17.990000,1280.260010,2704.290039,1346.500000,88.860001,29.525918
2011-01-21,9548500.0,18.469999,1283.349976,2689.540039,1341.000000,89.110001,29.397909
...,...,...,...,...,...,...,...
2022-08-26,16179500.0,25.559999,4057.659912,12141.709961,1736.099976,93.059998,39.029891
2022-08-29,13408900.0,26.209999,4030.610107,12017.669922,1736.599976,97.010002,39.040865
2022-08-30,15090600.0,26.209999,3986.159912,11883.139648,1723.199951,91.639999,39.063747


In [17]:
X = X.drop(labels="2022-09-01", axis=0)
X

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10
Date,,,,,,,
2011-01-14,5894200.0,15.460000,1293.239990,2755.300049,1360.400024,91.540001,29.558305
2011-01-18,8918200.0,15.870000,1295.020020,2765.850098,1368.099976,91.379997,29.633104
2011-01-19,12667700.0,17.309999,1281.920044,2725.360107,1370.199951,90.860001,29.594548
2011-01-20,15844900.0,17.990000,1280.260010,2704.290039,1346.500000,88.860001,29.525918
2011-01-21,9548500.0,18.469999,1283.349976,2689.540039,1341.000000,89.110001,29.397909
...,...,...,...,...,...,...,...
2022-08-25,15211900.0,21.780001,4199.120117,12639.269531,1757.699951,92.519997,39.054834
2022-08-26,16179500.0,25.559999,4057.659912,12141.709961,1736.099976,93.059998,39.029891
2022-08-29,13408900.0,26.209999,4030.610107,12017.669922,1736.599976,97.010002,39.040865


In [18]:
len(X)

2897

In [19]:
y = df["direction"]
y

Date
2011-01-14    0.0
2011-01-18    0.0
2011-01-19    0.0
2011-01-20    0.0
2011-01-21    1.0
             ... 
2022-08-26    0.0
2022-08-29    1.0
2022-08-30    0.0
2022-08-31    0.0
2022-09-01    1.0
Name: direction, Length: 2898, dtype: float64

In [20]:
y = y.drop(labels="2011-01-31", axis=0)
y

Date
2011-01-14    0.0
2011-01-18    0.0
2011-01-19    0.0
2011-01-20    0.0
2011-01-21    1.0
             ... 
2022-08-26    0.0
2022-08-29    1.0
2022-08-30    0.0
2022-08-31    0.0
2022-09-01    1.0
Name: direction, Length: 2897, dtype: float64

In [21]:
len(y)

2897

Split train test dataset

In [22]:
X_train = X.iloc[:int(len(X)*0.9),:]
X_train

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10
Date,,,,,,,
2011-01-14,5894200.0,15.460000,1293.239990,2755.300049,1360.400024,91.540001,29.558305
2011-01-18,8918200.0,15.870000,1295.020020,2765.850098,1368.099976,91.379997,29.633104
2011-01-19,12667700.0,17.309999,1281.920044,2725.360107,1370.199951,90.860001,29.594548
2011-01-20,15844900.0,17.990000,1280.260010,2704.290039,1346.500000,88.860001,29.525918
2011-01-21,9548500.0,18.469999,1283.349976,2689.540039,1341.000000,89.110001,29.397909
...,...,...,...,...,...,...,...
2021-06-30,12330500.0,15.830000,4297.500000,14503.950195,1770.800049,73.470001,59.312583
2021-07-01,11752100.0,15.480000,4319.939941,14522.379883,1775.900024,75.230003,59.215804
2021-07-02,9689700.0,15.070000,4352.339844,14639.330078,1782.599976,75.160004,59.235758


In [23]:
X_test = X.iloc[2598:2887,:]
X_test

,GM Volume,VIX,SP,NASDAQ,GOLD,OIL,rolling_10
Date,,,,,,,
2021-06-24,14445300.0,15.970000,4266.490234,14369.709961,1775.599976,73.300003,60.101782
2021-06-25,14255100.0,15.620000,4280.700195,14360.389648,1776.599976,74.050003,59.983052
2021-06-28,17152700.0,15.760000,4290.609863,14500.509766,1779.599976,72.910004,59.768542
2021-06-29,12171800.0,16.020000,4291.799805,14528.330078,1762.800049,72.980003,59.570993
2021-06-30,12330500.0,15.830000,4297.500000,14503.950195,1770.800049,73.470001,59.312583
...,...,...,...,...,...,...,...
2022-08-11,10221400.0,20.200001,4207.270020,12779.910156,1789.699951,94.339996,36.850866
2022-08-12,8905300.0,19.530001,4280.149902,13047.190430,1798.599976,92.089996,37.172132
2022-08-15,9626400.0,19.950001,4297.140137,13128.049805,1781.400024,89.410004,37.434534


In [24]:
y_train = y.iloc[:int(len(X)*0.9)]
y_train

Date
2011-01-14    0.0
2011-01-18    0.0
2011-01-19    0.0
2011-01-20    0.0
2011-01-21    1.0
             ... 
2021-07-01    0.0
2021-07-02    0.0
2021-07-06    0.0
2021-07-07    0.0
2021-07-08    0.0
Name: direction, Length: 2607, dtype: float64

In [25]:
y_test = y.iloc[2598:2887]
y_test

Date
2021-06-25    1.0
2021-06-28    0.0
2021-06-29    1.0
2021-06-30    1.0
2021-07-01    0.0
             ... 
2022-08-12    1.0
2022-08-15    0.0
2022-08-16    0.0
2022-08-17    0.0
2022-08-18    1.0
Name: direction, Length: 289, dtype: float64

Feature Selection

In [26]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [27]:
# StandardScaler
transfer = StandardScaler()
X_train_StandardScaler = transfer.fit_transform(X_train)
X_test_StandardScaler = transfer.transform(X_test)
pd.DataFrame(X_test_StandardScaler)

,0,1,2,3,4,5,6
0,0.067178,-0.235788,2.709874,2.922656,1.543686,0.221011,3.579607
1,0.042565,-0.283081,2.729101,2.919427,1.548051,0.252829,3.565719
2,0.417535,-0.264163,2.742510,2.967974,1.561147,0.204466,3.540626
3,-0.227029,-0.229032,2.744120,2.977613,1.487813,0.207436,3.517517
4,-0.206492,-0.254705,2.751833,2.969166,1.522734,0.228223,3.487289
...,...,...,...,...,...,...,...
284,-0.479425,0.335778,2.629744,2.371834,1.605235,1.113604,0.859767
285,-0.649737,0.245246,2.728357,2.464439,1.644085,1.018151,0.897348
286,-0.556422,0.301997,2.751346,2.492455,1.569004,0.904456,0.928043
287,0.002358,0.266865,2.762252,2.483620,1.533210,0.782276,0.961423


In [46]:
# MinMaxScaler
transfer = MinMaxScaler(feature_range=[2,3])
X_train_minmax = transfer.fit_transform(X_train)
X_test_minmax = transfer.transform(X_test)

# Build model 
# Model 1 Randon Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, plot_roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [30]:
def RandomForest(X_train,X_test, y_train, y_test):
    estimator = RandomForestClassifier(random_state=42)
    para_dict = {"max_depth": [5,8,15,25,30], "n_estimators":[120,200,300]}
    estimator = GridSearchCV(estimator, para_dict, cv = 3)
    estimator.fit(X_train,y_train)
    predicted = estimator.predict(X_test)

    #5:
    #print("confusion_matrix: \n",pd.DataFrame(confusion_matrix(y_test, predicted)))
    print("accuracy_score: \n", accuracy_score(y_test, predicted))
    print("\n")
    #6:
    print("best_parameters\n", estimator.best_params_)
    print("best_estimator_\n", estimator.best_estimator_)
    print("mean_squared_error",np.sqrt(mean_squared_error(y_test, predicted)))
    #print("best_score\n", estimator.best_score_)
    #print("cv_results_\n", estimator.cv_results_)
    return None

In [48]:
RandomForest(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

accuracy_score: 
 0.5363321799307958


best_parameters
 {'max_depth': 8, 'n_estimators': 120}
best_estimator_
 RandomForestClassifier(max_depth=8, n_estimators=120, random_state=42)
mean_squared_error 0.6809315825170721


In [50]:
RandomForest(X_train_minmax,X_test_minmax, y_train, y_test)

accuracy_score: 
 0.532871972318339


best_parameters
 {'max_depth': 8, 'n_estimators': 120}
best_estimator_
 RandomForestClassifier(max_depth=8, n_estimators=120, random_state=42)
mean_squared_error 0.6834676493307206


# Model 2 KNN next

In [32]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
def knn_(X_train,X_test, y_train, y_test):
    """
    Use KNN
    """
    #4:fit etimator
    estimator = KNeighborsClassifier(n_neighbors=11)
    estimator.fit(X_train, y_train)
    
    #5:model evaluation
    y_predict = estimator.predict(X_test)
    print("Accuracy Score\n", accuracy_score(y_test, y_predict))
    #print(confusion_matrix(y_predict, y_test))
    #print(pd.DataFrame(y_predict))
    return None

In [34]:
knn_(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

Accuracy Score
 0.5017301038062284


In [52]:
#Use MinMaxScaler have big improve
knn_(X_train_minmax,X_test_minmax, y_train, y_test)

Accuracy Score
 0.5259515570934256


# Model 3 XGboost

In [35]:
from xgboost import XGBClassifier

In [36]:
def xgb(X_train,X_test,y_train,y_test):
    xgboost = XGBClassifier(base_score=0.5, booster='gbtree')
    xgboost.fit(X_train, y_train)
    preds = xgboost.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print("accuracy:\n",accuracy_score(y_test, preds))
    print("RMSE: %f" % (rmse))

In [37]:
xgb(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

accuracy:
 0.5224913494809689
RMSE: 0.691020


In [53]:
#Use MinMaxScaler have big improve
xgb(X_train_minmax,X_test_minmax, y_train, y_test)

accuracy:
 0.5467128027681661
RMSE: 0.673266


In [38]:
def xgb_2(X_train,X_test,y_train,y_test):
    param_test1 = {
    'max_depth':[i for i in range(3,10,2)],
    'min_child_weight':[i for i in range(1,6,2)]}
    gsearch = GridSearchCV(
    estimator = XGBClassifier(
    learning_rate =0.1,
    n_estimators=140, max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27),
    param_grid = param_test1,
    scoring='roc_auc',
    n_jobs=4,
    cv=5)
    gsearch.fit(X_train,y_train)
    predicted = gsearch.predict(X_test)
    #print("confusion_matrix: \n",pd.DataFrame(confusion_matrix(y_test, predicted)))
    print("accuracy_score: \n", accuracy_score(y_test, predicted))
    print("\n")
    print("best_parameters\n", gsearch.best_params_)
    print("mean_squared_error",np.sqrt(mean_squared_error(y_test, predicted)))
    #print("best_score\n", gsearch.best_score_)
    #print("best_estimator_\n", gsearch.best_estimator_)
    #print("cv_results_\n", estimator.cv_results_)
    return None

In [40]:
xgb_2(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

accuracy_score: 
 0.5432525951557093


best_parameters
 {'max_depth': 3, 'min_child_weight': 3}
mean_squared_error 0.6758308995927093


In [54]:
xgb_2(X_train_minmax,X_test_minmax, y_train, y_test)

accuracy_score: 
 0.5432525951557093


best_parameters
 {'max_depth': 3, 'min_child_weight': 3}
mean_squared_error 0.6758308995927093


# Model 4 SVM

In [56]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
def svm(X_train,X_test,y_train,y_test):
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print("accuracy:\n",accuracy_score(y_test, predicted))
    return None

In [57]:
svm(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

accuracy:
 0.5224913494809689


In [58]:
svm(X_train_minmax,X_test_minmax, y_train, y_test)

accuracy:
 0.5224913494809689


In [43]:
def svm_cross_validation(X_train,X_test,y_train,y_test):    
    from sklearn.model_selection import GridSearchCV 
    model = SVC(kernel='rbf', probability=True)    
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}    
    grid_search = GridSearchCV(model, param_grid, n_jobs = 8, verbose=1)    
    grid_search.fit(X_train, y_train)    
    best_parameters = grid_search.best_estimator_.get_params()     
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)    
    model.fit(X_train, y_train)
    predicted = model.predict(X_test) 
    #print("confusion_matrix: \n",pd.DataFrame(confusion_matrix(y_test, predicted)))
    print("accuracy_score: \n", accuracy_score(y_test, predicted))
    return None

In [44]:
svm_cross_validation(X_train_StandardScaler,X_test_StandardScaler, y_train, y_test)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
accuracy_score: 
 0.5259515570934256


In [59]:
svm_cross_validation(X_train_minmax,X_test_minmax, y_train, y_test)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
accuracy_score: 
 0.4671280276816609


In [ ]:
# Summary
